Just generation with pretrained models

In [27]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
import re
import sys
import json

In [11]:
def readfile(fname):
    with open(fname, 'r') as f:
        return f.read().lower()

def preprocess(poem):
    # left only words
    poem = re.sub('[^!а-яіїєА-ЯІЇЄ\s\,\.\-\—\:\n\!\(\)\?’]', ' ', poem)
    return poem.replace('\t', '\n')

folder = './data/stus/'
# I've decided to use only one book to train
file = list(filter(lambda x: 'Palimpsesty' in x, os.listdir(folder)))[0]
raw_text = preprocess(readfile(folder + file))
chars = set(raw_text)
char_to_int =  { c:i for i, c in enumerate(chars) }
n_chars = len(raw_text)
n_vocab = len(chars)
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [1]:
!ls ./model/

weights-improvement-00-3.2714.hdf5  weights-improvement-10-2.8960.hdf5
weights-improvement-00-3.2963.hdf5  weights-improvement-11-2.8575.hdf5
weights-improvement-01-3.2196.hdf5  weights-improvement-12-2.8209.hdf5
weights-improvement-02-3.2023.hdf5  weights-improvement-13-2.7875.hdf5
weights-improvement-03-3.1715.hdf5  weights-improvement-14-2.7541.hdf5
weights-improvement-04-3.1364.hdf5  weights-improvement-15-2.7200.hdf5
weights-improvement-05-3.0861.hdf5  weights-improvement-16-2.6896.hdf5
weights-improvement-06-3.0415.hdf5  weights-improvement-17-2.6565.hdf5
weights-improvement-07-3.0031.hdf5  weights-improvement-18-2.6292.hdf5
weights-improvement-08-2.9682.hdf5  weights-improvement-19-2.6020.hdf5
weights-improvement-09-2.9310.hdf5


In [5]:
best_model_file = 'model/weights-improvement-19-2.6020.hdf5'

model = Sequential()
model.add(LSTM(256, input_shape=(100, 1)))
model.add(Dropout(0.2))
model.add(Dense(44, activation='softmax'))
model.load_weights(best_model_file)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [26]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"{}\"".format(''.join([int_to_char[value] for value in pattern])))
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:]
    
print("\nDone.")

Seed:
"ьше ні душі — тримавши вудку,
снував я довгу думу. передсвіт
ледь мерехтів над кручею. кричало
якесь"
а
--ф -щжжлхвж--ююяжюааю-
!ож-ю-ф!б!бяяяю--ббяяхжамф!яжле--- б
 --л!б!юаа
 --! б
в--ф ! ю-аюю
хаажгохгх-
!ц!й--
 -ф!бюяааг-
!--! б ю---
 -
!-аац -?!б-я!-!б!?!яжяю--ббяяюааю-
!о!--! ! л-мюя’а-ф -жж-щжоеях
-м!ю’ааж -ф!б-щ
жояяж--

KeyboardInterrupt: 